In [44]:
# Generalized Linear Multi Armed Bandit model
# ライブラリのインポート
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
import numpy.matlib
import scipy.linalg
import itertools
from numpy.random import *
from scipy import optimize
from scipy.stats import norm
import seaborn as sns

In [45]:
# パラメータの生成
k = 20   #パターン数
alpha = -1.25
beta = np.random.normal(0.0, 0.3, k)
thetat = np.append(alpha, beta) 

In [46]:
# 初期データを生成
# データの定義
validation = 10
d_id = np.repeat(np.arange(k+1), validation)
intercept = np.repeat(1.0, k+1)[:, np.newaxis]
factors = np.vstack((np.repeat(0.0, k), np.diag(np.repeat(1.0, k))))
get_x = np.hstack((intercept, factors))[d_id, ]

# 応答変数を生成
n = get_x.shape[0]
logit = np.dot(get_x, thetat)
Prob = np.exp(logit) / (1 + np.exp(logit))
get_y = np.random.binomial(1, Prob, n)

In [47]:
# Logistic linear multi armed bandit probremを解く 
# モデルをラプラス近似で推定するための関数
# モデルの対数事後分布の和
def loglike(theta, inv_Cov, y, x):
    # 対数尤度の計算
    logit_exp = np.exp(np.dot(x, theta))
    Prob = logit_exp / (1 + logit_exp)
    Loglike = y*np.log(Prob) + (1-y)*np.log(1-Prob)

    # 対数事前分布の計算
    Prior = -1/2 * np.dot(np.dot(theta, inv_Cov) * theta, np.repeat(1, k+1))
    
    # 対数事後分布の和
    Posterior = -(np.sum(Loglike) + Prior)
    return Posterior

# 対数事後分布の微分関数
def dll(theta, inv_Cov, y, x):
    #応答確率を計算
    logit_exp = np.exp(np.dot(x, theta))
    Prob = logit_exp / (1 + logit_exp)

    #勾配ベクトルとその和を計算
    dlogit = y[:, np.newaxis]*x - Prob[:, np.newaxis]*x
    dmvn = -np.dot(inv_Cov, theta.T).T
    LLd = -(np.sum(dlogit, axis=0) + dmvn)   #勾配の和
    return LLd

# 対数事後分布のヘシアン
def hessian(theta, inv_Cov, y, x):
    # 応答確率の計算
    logit_exp = np.exp(np.dot(x, theta))
    Prob_sq = logit_exp / np.power(1 + logit_exp, 2)

    # 対数事後分布の二階微分
    dlogit = -np.dot(x.T, Prob_sq[:, np.newaxis]*x)
    dmvn = -inv_Cov
    hessian = -(dlogit + dmvn)
    return hessian

In [48]:
# アルゴリズムの設定
R = 500
update = 4
disp = 50
keep = 1
dl = 100   #EMアルゴリズムの対数尤度の差の初期値
tol = 0.1
a = 10.0

# パラメータの格納用配列
THETA = np.zeros((int(R/keep), k+1))
PROB = np.repeat(0.0, int(R/keep))
X = np.zeros((int(R/keep), k+1))

In [49]:
# 初期値の設定
Cov = np.diag(np.append(100.0, np.repeat(1.0, k)))
inv_Cov = np.linalg.inv(Cov)
theta = np.append(-1.0, np.random.normal(0.0, 0.3, k))
y = get_y.copy()
x = get_x.copy()

# ニュートンCG法でパラメータの初期値を設定
# パラメータを初期値を設定
n = x.shape[0]
index_n = np.arange(n)
eta = a * 1/n

# パラメータを推定
res = optimize.minimize(loglike, theta, jac=dll, hess=hessian, args=(inv_Cov, y, x), method="Newton-CG")
theta = res.x
H = np.linalg.inv(hessian(theta, inv_Cov, y, x))   #ヘシアンの逆行列
LL = res.fun

In [50]:
# トンプソン抽出で最適な組み合わせを取得
for rp in range(R):
    # 新しいパラメータと入力データを生成
    new_theta = np.random.multivariate_normal(theta, H, 1).reshape(-1)
    new_x = np.full((validation, k+1), np.append(1.0, np.array(new_theta[1:] > 0, dtype="float")))

    # 検証結果データを生成
    logit = np.dot(new_x, thetat)
    Prob = np.exp(logit) / (1 + np.exp(logit))
    new_y = np.random.binomial(1, Prob, validation)

    # データを結合
    y = np.append(y, new_y)
    x = np.vstack((x, new_x))

    # ニュートンCG法でパラメータのを更新
    res = optimize.minimize(loglike, theta, jac=dll, hess=hessian, args=(inv_Cov, y, x), method="dogleg")
    LL = res.fun
    theta = res.x
    H = np.linalg.inv(hessian(theta, inv_Cov, y, x))   #ヘシアンの逆行列

    # パラメータの格納
    if rp%keep==0:
        mkeep = int(rp/keep)
        THETA[mkeep, ] = theta
        PROB[mkeep] = Prob[0, ]
        X[mkeep, ] = new_x[0, ]
    
    # パラメータを表示
    if rp%disp==0:
        print(rp)
        print(np.round(np.vstack((theta, thetat)), 2))
        print(new_x[0, ])

0
[[-1.57  0.81 -0.32 -0.32 -0.32  0.48 -0.83  0.11  0.11  0.48  0.81 -0.32
  -0.83  0.11 -0.32  0.11  0.81 -0.32  0.11  0.11 -0.83]
 [-1.25  0.23  0.08 -0.24 -0.31  0.41 -0.01  0.25  0.11  0.03  0.41  0.13
  -0.07  0.78 -0.17  0.45  0.38  0.3  -0.4   0.17 -0.11]]
[1. 1. 1. 1. 0. 1. 0. 1. 1. 1. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0.]
50
[[-1.68  0.55  0.02 -0.3  -0.36  0.23 -0.15 -0.   -0.14 -0.12  1.03  0.35
  -0.37  0.55  0.05  0.44  0.27  0.29 -0.04 -0.05 -0.24]
 [-1.25  0.23  0.08 -0.24 -0.31  0.41 -0.01  0.25  0.11  0.03  0.41  0.13
  -0.07  0.78 -0.17  0.45  0.38  0.3  -0.4   0.17 -0.11]]
[1. 1. 1. 0. 0. 1. 1. 1. 0. 0. 1. 1. 0. 1. 0. 1. 1. 1. 0. 1. 1.]
100
[[-1.69  0.5   0.06 -0.34 -0.29  0.32 -0.09  0.15 -0.05  0.07  0.85  0.24
  -0.45  0.66  0.02  0.4   0.25  0.54 -0.32 -0.19 -0.23]
 [-1.25  0.23  0.08 -0.24 -0.31  0.41 -0.01  0.25  0.11  0.03  0.41  0.13
  -0.07  0.78 -0.17  0.45  0.38  0.3  -0.4   0.17 -0.11]]
[1. 1. 0. 0. 0. 1. 0. 1. 0. 1. 1. 1. 0. 1. 0. 1. 1. 1. 0. 1. 0.]
150
[[-